# Learn Python the Hard Way — Exercises 46–52
## Week 9: Project Structure, Testing & Web Applications

Source: [笨方法學Python (Learn Python the Hard Way CN)](https://flyouting.gitbooks.io/learn-python-the-hard-way-cn/content/introduction.html)

**Why here (Week 9 — Vibe Coding & Claude Code)?**  
These exercises cover the professional foundations of Python development: project layout, automated testing, and web apps — exactly the skills you need when building real projects with AI coding tools.

**Topics covered:**
- Project skeleton & directory structure (`setup.py`, packages)
- Automated testing with `unittest`
- Parsing complex user input
- Writing code phrases
- Your first web application with `flask` (or `lpthw.web`)
- Getting input from a browser
- Starting your web game

---

## Exercise 46 — A Project Skeleton (專案骨架)

**Concept:** Every Python project should follow a standard layout. This makes it installable, testable, and shareable.

### Standard layout
```
my_project/
├── my_project/           # The actual package (Python source)
│   ├── __init__.py       # Makes this folder a Python package
│   └── main.py
├── tests/                # Test suite
│   ├── __init__.py
│   └── test_main.py
├── setup.py              # Installation script
├── README.md             # Documentation
└── requirements.txt      # Dependencies
```

In [ ]:
import os

# Create a project skeleton programmatically
project_name = "ex46_skeleton"

dirs = [
    f"{project_name}/{project_name}",
    f"{project_name}/tests",
    f"{project_name}/docs",
]

for d in dirs:
    os.makedirs(d, exist_ok=True)
    print(f"Created: {d}/")

# Create __init__.py files
for init in [f"{project_name}/{project_name}/__init__.py",
             f"{project_name}/tests/__init__.py"]:
    with open(init, 'w') as f:
        pass
    print(f"Created: {init}")

# Create setup.py
setup_content = '''\
try:
    from setuptools import setup
except ImportError:
    from distutils.core import setup

config = {
    'description': 'My project',
    'author': 'Your Name',
    'author_email': 'your@email.com',
    'version': '0.1',
    'install_requires': [],
    'packages': ['NAME'],
    'name': 'NAME',
}

setup(**config)
'''
with open(f"{project_name}/setup.py", 'w') as f:
    f.write(setup_content)
print(f"Created: {project_name}/setup.py")

print("\nProject skeleton created!")

In [ ]:
# Verify the structure
for root, dirs_found, files in os.walk(project_name):
    level = root.replace(project_name, '').count(os.sep)
    indent = '  ' * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = '  ' * (level + 1)
    for file in files:
        print(f"{sub_indent}{file}")

**Study Drills:**
1. What is `__init__.py`? What happens if it's missing from a package folder?
2. Create a real project using this structure for your next assignment.
3. Look up `pyproject.toml` — the modern alternative to `setup.py`.

---
## Exercise 47 — Automated Testing (自動化測試)

**Concept:** Write tests **before** or alongside your code. A test function runs your code and asserts the output is what you expected. `unittest` is Python's built-in testing framework.

In [ ]:
# The function we want to test
def add(a, b):
    return a + b

def subtract(a, b):
    return a - b

def multiply(a, b):
    return a * b

def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

In [ ]:
import unittest

class TestMathFunctions(unittest.TestCase):

    def test_add(self):
        self.assertEqual(add(1, 1), 2)
        self.assertEqual(add(0, 0), 0)
        self.assertEqual(add(-1, 1), 0)

    def test_subtract(self):
        self.assertEqual(subtract(5, 3), 2)
        self.assertEqual(subtract(0, 5), -5)

    def test_multiply(self):
        self.assertEqual(multiply(3, 4), 12)
        self.assertEqual(multiply(-2, 5), -10)

    def test_divide(self):
        self.assertAlmostEqual(divide(10, 2), 5.0)
        self.assertAlmostEqual(divide(7, 2), 3.5)

    def test_divide_by_zero(self):
        with self.assertRaises(ValueError):
            divide(10, 0)


# Run tests in a notebook
loader = unittest.TestLoader()
suite = loader.loadTestsFromTestCase(TestMathFunctions)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

**Common `unittest` assertions:**

| Method | Tests |
|--------|-------|
| `assertEqual(a, b)` | `a == b` |
| `assertNotEqual(a, b)` | `a != b` |
| `assertTrue(x)` | `bool(x)` is True |
| `assertFalse(x)` | `bool(x)` is False |
| `assertAlmostEqual(a, b)` | `a ≈ b` (floats) |
| `assertRaises(Error, func, args)` | `func(args)` raises `Error` |
| `assertIn(a, b)` | `a in b` |

**Study Drills:**
1. Break the `add` function intentionally (e.g., `return a - b`) and observe the test failure.
2. Add tests for edge cases: very large numbers, floats, strings.
3. Look up `pytest` — a more popular modern testing framework. How does it compare?

---
## Exercise 48 — Advanced User Input (更復雜的使用者輸入)

**Concept:** Parsing user input with direction/action/object patterns, like a text adventure game parser. Split input into tokens and match against known commands.

In [ ]:
# A simple lexer/parser for text adventure input

DIRECTIONS = set(['north', 'south', 'east', 'west', 'up', 'down',
                  'left', 'right', 'back'])

STOP_WORDS = set(['the', 'in', 'of', 'from', 'at', 'it', 'a', 'an'])

ACTIONS = set(['go', 'take', 'drop', 'look', 'examine', 'pick', 'attack',
               'kill', 'open', 'close', 'use', 'give', 'eat', 'quit'])


def scan(sentence):
    """Tokenize a sentence into (type, value) tuples."""
    results = []
    words = sentence.strip().lower().split()

    for word in words:
        if word in STOP_WORDS:
            results.append(('stop', word))
        elif word in DIRECTIONS:
            results.append(('direction', word))
        elif word in ACTIONS:
            results.append(('verb', word))
        else:
            try:
                results.append(('number', int(word)))
            except ValueError:
                results.append(('noun', word))

    return results


# Test the scanner
test_inputs = [
    "go north",
    "take the lamp",
    "look at the bear",
    "open door",
    "kill the dragon with sword",
    "go 3 south",
]

for sentence in test_inputs:
    result = scan(sentence)
    print(f"{sentence!r:40s} -> {result}")

In [ ]:
# Test with unittest
import unittest

class TestScan(unittest.TestCase):

    def test_directions(self):
        result = scan("go north")
        self.assertIn(('direction', 'north'), result)

    def test_verbs(self):
        result = scan("take the lamp")
        self.assertIn(('verb', 'take'), result)

    def test_stop_words_removed(self):
        result = scan("take the lamp")
        token_types = [t[0] for t in result]
        self.assertIn('stop', token_types)

    def test_number(self):
        result = scan("go 3 north")
        self.assertIn(('number', 3), result)


loader = unittest.TestLoader()
suite = loader.loadTestsFromTestCase(TestScan)
unittest.TextTestRunner(verbosity=2).run(suite)

**Study Drills:**
1. Add support for adjectives (e.g., "old", "large", "golden").
2. What happens with unknown words? Is `'noun'` the right category?
3. Write a `parse()` function that takes the token list and returns an action dictionary: `{'verb': 'take', 'noun': 'lamp'}`.

---
## Exercise 49 — Making Sentences (寫程式碼語句)

**Concept:** Build a parser that takes the token list from Exercise 48 and produces structured sentence objects (verb, subject, object).

In [ ]:
class ParserError(Exception):
    """Raised when the parser cannot understand the input."""
    pass


class Sentence(object):
    def __init__(self, subject, verb, obj):
        self.subject = subject  # (type, word)
        self.verb = verb        # (type, word)
        self.obj = obj          # (type, word)

    def __repr__(self):
        return f"Sentence(subject={self.subject}, verb={self.verb}, object={self.obj})"


def peek(word_list):
    """Look at the type of the next token without consuming it."""
    if word_list:
        word = word_list[0]
        return word[0]
    return None


def match(word_list, expecting):
    """Consume and return the next token if it matches the expected type."""
    if word_list:
        word = word_list.pop(0)
        if word[0] == expecting:
            return word
        raise ParserError(f"Expected {expecting!r}, got {word[0]!r}: {word[1]!r}")
    raise ParserError("Expected more input but got nothing.")


def skip(word_list, token_type):
    """Skip all tokens of the given type."""
    while peek(word_list) == token_type:
        match(word_list, token_type)


def parse_verb(word_list):
    skip(word_list, 'stop')
    if peek(word_list) == 'verb':
        return match(word_list, 'verb')
    raise ParserError(f"Expected a verb, got: {peek(word_list)!r}")


def parse_object(word_list):
    skip(word_list, 'stop')
    next_token = peek(word_list)
    if next_token == 'noun':
        return match(word_list, 'noun')
    if next_token == 'direction':
        return match(word_list, 'direction')
    raise ParserError(f"Expected noun/direction, got: {next_token!r}")


def parse_subject(word_list, subj):
    verb = parse_verb(word_list)
    obj = parse_object(word_list)
    return Sentence(subj, verb, obj)


def parse_sentence(word_list):
    skip(word_list, 'stop')
    start = peek(word_list)

    if start == 'noun':
        subj = match(word_list, 'noun')
        return parse_subject(word_list, subj)
    elif start == 'verb':
        return parse_subject(word_list, ('noun', 'player'))
    else:
        raise ParserError(f"Must start with noun or verb, not {start!r}")


# Test it
from copy import deepcopy

tests = [
    "go north",
    "take the lamp",
    "bear eat honey",
]

for text in tests:
    tokens = scan(text)
    try:
        sentence = parse_sentence(deepcopy(tokens))
        print(f"{text!r:30s} -> {sentence}")
    except ParserError as e:
        print(f"{text!r:30s} -> ERROR: {e}")

**Study Drills:**
1. Add tests for `parse_sentence` using `unittest`.
2. What happens with `"the go north"`? Why?
3. Extend the parser to handle adjectives before nouns.

---
## Exercise 50 — Your First Website (你的第一個網站)

**Concept:** Build a minimal web application using Flask. A web server listens for HTTP requests and returns HTML responses.

> Install Flask first: `pip install flask`

In [ ]:
# Check if Flask is installed
try:
    import flask
    print(f"Flask version: {flask.__version__}")
except ImportError:
    print("Flask not installed. Run: pip install flask")

In [ ]:
# Save this as a .py file and run it: python3 ex50_app.py
# Then open http://localhost:5000 in your browser

flask_app_code = '''
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return \'\'\'<html>
<head><title>Hello Web</title></head>
<body>
  <h1>Hello from Flask!</h1>
  <p>My first web app.</p>
  <ul>
    <li><a href="/hello">Say Hello</a></li>
    <li><a href="/about">About</a></li>
  </ul>
</body>
</html>\'\'\'

@app.route('/hello')
def hello():
    return \'<h1>Hello, World!</h1>\'

@app.route('/about')
def about():
    return \'<h1>About</h1><p>Built with Flask.</p>\'

if __name__ == "__main__":
    app.run(debug=True)
'''

with open("ex50_app.py", "w") as f:
    f.write(flask_app_code)

print("Saved to ex50_app.py")
print("Run with: python3 ex50_app.py")
print("Then open: http://localhost:5000")

**Key Flask concepts:**
- `@app.route('/')` — maps a URL path to a Python function
- The function returns HTML as a string
- `debug=True` — auto-reloads when you save changes

**Study Drills:**
1. Run `ex50_app.py` from your terminal and visit each URL.
2. Add a `/contact` route.
3. What is a **decorator** (`@app.route`)? Search "Python decorators".

---
## Exercise 51 — Getting Input from a Browser (從瀏覽器獲取輸入)

**Concept:** HTML forms send data to your Flask server via GET or POST requests. `request.args` (GET) or `request.form` (POST) contains the submitted data.

In [ ]:
flask_form_code = '''
from flask import Flask, request

app = Flask(__name__)

# HTML form template
FORM_HTML = \'\'\'<html>
<head><title>Hello Form</title></head>
<body>
  <form method="GET" action="/hello">
    <label>Your name: <input type="text" name="name"></label><br>
    <label>Your greeting: <input type="text" name="greeting"></label><br>
    <input type="submit" value="Greet!">
  </form>
</body>
</html>\'\'\'

@app.route("/")
def index():
    return FORM_HTML

@app.route("/hello")
def hello():
    name = request.args.get(\'name\', \'World\')
    greeting = request.args.get(\'greeting\', \'Hello\')
    return f\'<h1>{greeting}, {name}!</h1><a href="/">Back</a>\'

if __name__ == "__main__":
    app.run(debug=True, port=5001)
'''

with open("ex51_app.py", "w") as f:
    f.write(flask_form_code)

print("Saved to ex51_app.py")
print("Run: python3 ex51_app.py")
print("Open: http://localhost:5001")

**Study Drills:**
1. Change `method="GET"` to `method="POST"` and update the Flask route to use `request.form`.
2. What appears in the browser URL bar for GET vs POST? Why does this matter?
3. Add a text area (`<textarea>`) to the form.

---
## Exercise 52 — The Start of Your Web Game (開始你的web遊戲)

**Concept:** Combine the OOP game from Exercise 45 with the Flask web server from Exercises 50–51 to create a browser-playable text adventure.

In [ ]:
web_game_code = '''
from flask import Flask, request, session

app = Flask(__name__)
app.secret_key = \'your-secret-key\'   # needed for sessions

# --- Game world ---
ROOMS = {
    \'entrance\': {
        \'name\': \'Entrance Hall\',
        \'description\': \'A dimly lit hall. Cobwebs hang from the ceiling.\',
        \'exits\': {{\'north\': \'library\', \'east\': \'garden\'}},
        \'items\': [],
    },
    \'library\': {
        \'name\': \'Library\',
        \'description\': \'Shelves of ancient books surround you.\',
        \'exits\': {{\'south\': \'entrance\'}},
        \'items\': [\'old map\', \'dusty tome\'],
    },
    \'garden\': {
        \'name\': \'Garden\',
        \'description\': \'Sunlight streams through the overgrown garden.\',
        \'exits\': {{\'west\': \'entrance\'}},
        \'items\': [\'golden key\'],
    },
}

def render_room(room_id, inventory, message=""):
    room = ROOMS[room_id]
    exits_html = " | ".join(
        f\'<a href="/move/{direction}">{direction.capitalize()}</a>\'
        for direction in room[\'exits\']
    )
    items_html = ", ".join(room[\'items\']) or "nothing"
    inv_html = ", ".join(inventory) or "nothing"
    take_html = "".join(
        f\'<a href="/take/{item}">[take {item}]</a> \'
        for item in room[\'items\']
    )
    return f\'\'\'<html><body>
    <h2>{room[\'name\']}</h2>
    <p>{room[\'description\']}</p>
    {f\'<p style="color:green">{message}</p>\' if message else ""}
    <p><strong>You see:</strong> {items_html}  {take_html}</p>
    <p><strong>Inventory:</strong> {inv_html}</p>
    <p><strong>Exits:</strong> {exits_html}</p>
    </body></html>\'\'\'

@app.route("/")
def index():
    session[\'room\'] = \'entrance\'
    session[\'inventory\'] = []
    return render_room(\'entrance\', [])

@app.route("/move/<direction>")
def move(direction):
    room_id = session.get(\'room\', \'entrance\')
    inventory = session.get(\'inventory\', [])
    room = ROOMS[room_id]
    if direction in room[\'exits\']:
        session[\'room\'] = room[\'exits\'][direction]
        return render_room(session[\'room\'], inventory)
    return render_room(room_id, inventory, message="You can\'t go that way.")

@app.route("/take/<item>")
def take(item):
    room_id = session.get(\'room\', \'entrance\')
    inventory = session.get(\'inventory\', [])
    room = ROOMS[room_id]
    if item in room[\'items\']:
        room[\'items\'].remove(item)
        inventory.append(item)
        session[\'inventory\'] = inventory
        return render_room(room_id, inventory, message=f"You picked up: {item}")
    return render_room(room_id, inventory, message=f"There is no {item} here.")

if __name__ == "__main__":
    app.run(debug=True, port=5002)
'''

with open("ex52_webgame.py", "w") as f:
    f.write(web_game_code)

print("Saved to ex52_webgame.py")
print("Run: python3 ex52_webgame.py")
print("Open: http://localhost:5002")

**Study Drills:**
1. Run the web game and play through it.
2. Add a win condition: if the player is in `library` with `golden key` in inventory, show a victory page.
3. Add CSS styling to make it look less plain.
4. What is a **session** in Flask? How does it store state between requests?

---

## Summary

| Exercise | Core Concept |
|----------|--------------|
| 46 | Project directory structure, `__init__.py`, `setup.py` |
| 47 | `unittest` — automated tests, assertions, `assertRaises` |
| 48 | Lexer / tokenizer — parsing user input into tokens |
| 49 | Parser — building structured `Sentence` objects from tokens |
| 50 | Flask — `@app.route`, returning HTML from Python functions |
| 51 | HTML forms — GET/POST, `request.args`, `request.form` |
| 52 | Full web game — sessions, navigation, item pickup |

**Connection to AI/vibe coding:**
- The project skeleton (Ex 46) is what Claude Code generates for you automatically
- Tests (Ex 47) let Claude verify its own changes — test-driven development pairs well with AI coding
- Flask (Ex 50–52) is the foundation for AI-powered web demos and APIs